In [ ]:
trainData['Timestamp'] = pd.to_datetime(trainData['Timestamp'])
trainTrips = trainData['trips']

model = ExponentialSmoothing(trainTrips, trend='add', seasonal='add', seasonal_periods=24)

In [ ]:
#exponential smoothing model 3/9

import pandas as pd
from statsmodels.tsa.api import ExponentialSmoothing
import plotly.express as px
import numpy as np


trainData = pd.read_csv("assignment_data_train.csv")
testData = pd.read_csv("assignment_data_test.csv")

trainData['Timestamp'] = pd.to_datetime(trainData['Timestamp'])
trainTrips = trainData['trips']

model = ExponentialSmoothing(trainTrips, trend='add',
    seasonal='add', seasonal_periods=24)

modelFit = model.fit(optimized = True)

forecast1 = len(testData)
pred = modelFit.forecast(forecast1)
pred = np.array(pred)

In [ ]:
#VAR model 9/9

import pandas as pd
from statsmodels.tsa.api import VAR
import plotly.express as px
import numpy as np


trainData = pd.read_csv("assignment_data_train.csv")
testData = pd.read_csv("assignment_data_test.csv")

trainData['Timestamp'] = pd.to_datetime(trainData['Timestamp'])
trainData.set_index(pd.DatetimeIndex(trainData['Timestamp']), inplace=True)

varData = trainData[['trips','hour']].dropna()[:-50]
model = VAR(varData)

lag = model.select_order()
print(lag.summary())

modelFit = model.fit(lag.aic)

nPeriods = varData.values[-modelFit.k_ar:]
pred = modelFit.forecast(nPeriods, steps = 31*24)
pred = pred[:,0]

In [ ]:
#Prophet model 0/9 (not allowed in test)

import pandas as pd
import numpy as np
from prophet import Prophet


trainData = pd.read_csv("assignment_data_train.csv")
testData = pd.read_csv("assignment_data_test.csv")

trainData['Timestamp'] = pd.to_datetime(trainData['Timestamp'])
trainData1 = trainData[['Timestamp', 'trips']]

trainData1 = pd.DataFrame(trainData1.values, columns = ['ds', 'y'])

model = Prophet(changepoint_prior_scale=0.5, daily_seasonality=True, weekly_seasonality=True)
modelFit = model.fit(trainData1)

future = modelFit.make_future_dataframe(periods= 744, freq= 'h')
pred = modelFit.predict(future)

pred = pred['yhat'][-744:]
pred = np.array(pred)

In [ ]:
#GAM model - seems too detailed for this, did not complete

from pygam import LinearGAM, s, f, l
import patsy as pt

import pandas as pd
import numpy as np
from plotly import tools
import plotly.offline as py
import plotly.graph_objs as go
from plotly.subplots import make_subplots


trainData = pd.read_csv("assignment_data_train.csv")
testData = pd.read_csv("assignment_data_test.csv")

#generate x and y matrices
eqn = """trips ~ -1 + year + month + day + hour"""
y, x = pt.dmatrices(eqn, data = trainData)

#initializing and fit the model
gam = LinearGAM(l(0) + f(1) + f(2) + s(3))
gam = gam.gridsearch(np.asarray(x), y)

titles = ['year', 'month', 'day', 'hour']
fig = make_subplots(rows = 2, cols = 3, subplot_titles = titles)
fig['layout'].update(height=800, width=1200, title='pyGAM', showlegend=False)

for i, title in enumerate(titles):
    XX = gam.generate_X_grid(term=i)
    pdep, confi = gam.partial_dependence(term=i, width=0.95)
    trace = go.Scatter(x = XX[:,i], y=pdep, mode='lines', name='Effect')
    ci1 = go.Scatter(x = XX[:,i], y=confi[:,0], line = dict(dash='dash', color='grey'), name='95% CI')
    ci2 = go.Scatter(x = XX[:,i], y=confi[:,1], line = dict(dash='dash', color='grey'), name='95% CI')

    if i<3:
        fig.append_trace(trace, 1, i+1)
        fig.append_trace(ci1, 1, i+1)
        fig.append_trace(ci2, 1, i+1)
    else:
        fig.append_trace(trace, 2, i-2)
        fig.append_trace(ci1, 2, i-2)
        fig.append_trace(ci2, 2, i-2)

py.iplot(fig)

In [ ]:
#chatgpt model for VARMA(x) to compare from my previous code that completed the assignment

import pandas as pd
import numpy as np
from statsmodels.tsa.statespace.varmax import VARMAX

# Load data
trainData = pd.read_csv("assignment_data_train.csv")
testData = pd.read_csv("assignment_data_test.csv")

trainData['Timestamp'] = pd.to_datetime(trainData['Timestamp'])
testData['Timestamp'] = pd.to_datetime(testData['Timestamp'])

# Endogenous variables (VAR part)
endog = trainData[['trips', 'hour']]

# Exogenous variables (X part)
exog = trainData[['month', 'day']]

# --------------------
# Define model
# --------------------
model = VARMAX(
    endog,
    exog=exog,
    order=(1, 1),      # VARMA(1,1)
    trend='c'
)

# Fit model
modelFit = model.fit(disp=False)

# --------------------
# Forecast January
# --------------------
exog_future = testData[['month', 'day']]

forecast = modelFit.forecast(
    steps=744,
    exog=exog_future
)

# Extract trips forecast only
pred = forecast['trips'].values